In [1]:
import script
from flask import Flask, jsonify, request
import pandas as pd

In [2]:
app = Flask(__name__)

In [3]:
"""@app.route('/api/recette', methods = ['GET'])
def get_recette_all():
    args = request.args
    id = args.get("id")
    if id is None:
        df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")
        list_all = list(df['id'])
        df1 = []
        for i in list_all:
            recipe = script.SmallRecipe(int(i))
            df1.append(recipe.__dict__())
        return {'recipes': df}, 200
    recipe = script.SmallRecipe(int(id))
    return {'recipes': recipe.__dict__()}, 200"""

'@app.route(\'/api/recette\', methods = [\'GET\'])\ndef get_recette_all():\n    args = request.args\n    id = args.get("id")\n    if id is None:\n        df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")\n        list_all = list(df[\'id\'])\n        df1 = []\n        for i in list_all:\n            recipe = script.SmallRecipe(int(i))\n            df1.append(recipe.__dict__())\n        return {\'recipes\': df}, 200\n    recipe = script.SmallRecipe(int(id))\n    return {\'recipes\': recipe.__dict__()}, 200'

In [4]:
@app.route('/api/recette', methods = ['GET'])
def get_recette_all():
    args = request.args
    id = args.get("id")
    if id is None:
        df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")
        id = list(df['id'])
        df1 = []
        for i in id:
            recipe = script.SmallRecipe(int(i))
            df1.append(recipe.__dict__())
        return {'recipes': df1}, 200
    recipe = script.SmallRecipe(int(id))
    return {'recipes': recipe.__dict__()}, 200

In [5]:
@app.route('/api/machine_learning', methods = ['GET'])
def get_machine_learning():
    args = request.args
    id = args.get("id")
    recipe_list = script.main_function(int(id))
    df1 = []
    for i in recipe_list:
        suggestion = script.SmallRecipe(int(i))
        df1.append(suggestion.__dict__())
    return {'suggestion': df1}, 200


In [6]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jul/2022 21:17:11] "GET /api/machine_learning?id=1533 HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:21:11] "GET /api/machine_learning?id=1533 HTTP/1.1" 200 -
